In [1]:
from health_lifestyle_diabetes.infrastructure.utils.paths import get_repository_root
from health_lifestyle_diabetes.infrastructure.utils.config_loader import YamlConfigLoader
root = get_repository_root()

# Load configuration
config_loader = YamlConfigLoader()
paths = config_loader.load_config(f"{root}/configs/paths.yaml")

# 
print(paths)

{'data': {'root': 'data/', 'input': {'raw_dataset': 'data/raw/diabetes_health_indicators.csv', 'train_dataset': 'data/input/train.csv', 'test_dataset': 'data/input/test.csv'}, 'processed': {'cleaned_dataset': 'data/processed/cleaned_diabetes.csv', 'features_dataset': 'data/processed/features_diabetes.csv'}, 'output': {'model': 'data/output/model.pkl', 'pipeline': 'data/output/pipeline.pkl'}}, 'reports': {'eda_report': 'reports/eda_reports', 'metrics_report': 'reports/metrics', 'curves_reports': 'reports/figures', 'cm_reports': 'reports/cm'}, 'logs': {'folder': 'logs', 'main_log': 'logs/app.log'}}


In [2]:
from health_lifestyle_diabetes.infrastructure.data_sources.csv_dataset_repository import CSVDatasetRepository 
from health_lifestyle_diabetes.infrastructure.logger.loguru_logger import LoguruLogger

logger = LoguruLogger()

# Load Dataset fron CSV file
csv_repo = CSVDatasetRepository(logger=logger,) 

# Lit le fichier CSV et le charge en mémoire (DataFrame Pandas).
data = csv_repo.load_dataset()

# Show the 5 first rows
display(data.head()) # Affiche les 5 premières lignes pour validation.
print(f"le dataset chargé a {data.shape[0]} lignes et {data.shape[1]} colonnes")

2026-01-08 02:45:55.534 | INFO     | health_lifestyle_diabetes.infrastructure.data_sources.csv_dataset_repository:load_dataset:54 - Chargement du dataset depuis : /Users/surelmanda/Downloads/ml-projects/Clean-Architecture-MLops/health_lifestyle_diabetes/data/raw/diabetes_health_indicators.csv
2026-01-08 02:45:55.683 | INFO     | health_lifestyle_diabetes.infrastructure.data_sources.csv_dataset_repository:load_dataset:61 - Dataset chargé avec succès (97297 lignes, 31 colonnes).


,Age,gender,ethnicity,education_level,income_level,employment_status,smoking_status,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,...,hdl_cholesterol,ldl_cholesterol,triglycerides,glucose_fasting,glucose_postprandial,insulin_level,hba1c,diabetes_risk_score,diabetes_stage,diagnosed_diabetes
0,58,Male,Asian,Highschool,Lower-Middle,Employed,Never,0,215,5.7,...,41,160,145,136,236,6.36,8.18,29.6,Type 2,1
1,52,Female,White,Highschool,Middle,Employed,Former,1,143,6.7,...,55,50,30,93,150,2.00,5.63,23.0,No Diabetes,0
2,60,Male,Hispanic,Highschool,Middle,Unemployed,Never,1,57,6.4,...,66,99,36,118,195,5.07,7.51,44.7,Type 2,1
3,74,Female,Black,Highschool,Low,Retired,Never,0,49,3.4,...,50,79,140,139,253,5.28,9.03,38.2,Type 2,1
4,46,Male,White,Graduate,Middle,Retired,Never,1,109,7.2,...,52,125,160,137,184,12.74,7.20,23.5,Type 2,1


le dataset chargé a 97297 lignes et 31 colonnes


In [3]:
from health_lifestyle_diabetes.infrastructure.utils.config_loader import YamlConfigLoader
from health_lifestyle_diabetes.infrastructure.utils.paths import get_repository_root
from health_lifestyle_diabetes.infrastructure.logger.loguru_logger import LoguruLogger
from health_lifestyle_diabetes.infrastructure.splitters.sklearn_splitter import SklearnDatasetSplitter

root = get_repository_root()

# Load configuration
config_loader = YamlConfigLoader()
cfg = config_loader.load_config(f"{root}/configs/splitter.yaml")
print(cfg)

logger = LoguruLogger()

splitter = SklearnDatasetSplitter(
    train_size=cfg["splitter"]["train_size"],
    target_column=cfg["splitter"]["target_column"],
    random_state=cfg["splitter"]["random_state"],
    logger=logger,
)
train_df, test_df = splitter.split(data)

2026-01-08 02:45:56.576 | DEBUG    | health_lifestyle_diabetes.infrastructure.splitters.sklearn_splitter:__init__:37 - SklearnDatasetSplitter initialized
2026-01-08 02:45:56.577 | INFO     | health_lifestyle_diabetes.infrastructure.splitters.sklearn_splitter:split:42 - Starting dataset split
2026-01-08 02:45:56.605 | INFO     | health_lifestyle_diabetes.infrastructure.splitters.sklearn_splitter:split:57 - Split done | train=(82702, 31) | test=(14595, 31)


{'splitter': {'train_size': 0.85, 'target_column': 'diagnosed_diabetes', 'random_state': 42}}


In [4]:
from pathlib import Path
from health_lifestyle_diabetes.infrastructure.logger.loguru_logger import LoguruLogger
from health_lifestyle_diabetes.infrastructure.data_sources.csv_dataset_repository import CSVDatasetRepository 

csv_repo = CSVDatasetRepository(logger=logger,) 

logger = LoguruLogger()

output_paths = {
    "train": get_repository_root() / Path(paths["data"]["input"]["train_dataset"]),
    "test": get_repository_root() / Path(paths["data"]["input"]["test_dataset"]),
}

datasets = {
    "train": train_df,
    "test": test_df,
}

for split_name, dataset in datasets.items():
    output_path = output_paths[split_name]

    logger.info(
        f"Saving {split_name} dataset | "
        f"rows={dataset.shape[0]} | "
        f"columns={dataset.shape[1]}"
    )

    csv_repo.save_dataset(dataset, output_path)

    logger.info(
        f"{split_name.capitalize()} dataset successfully saved at '{output_path}'"
    )

2026-01-08 02:45:56.612 | INFO     | __main__:<module>:22 - Saving train dataset | rows=82702 | columns=31
2026-01-08 02:45:56.612 | INFO     | health_lifestyle_diabetes.infrastructure.data_sources.csv_dataset_repository:save_dataset:74 - Sauvegarde du dataset dans : /Users/surelmanda/Downloads/ml-projects/Clean-Architecture-MLops/health_lifestyle_diabetes/data/input/train.csv
2026-01-08 02:45:57.060 | INFO     | health_lifestyle_diabetes.infrastructure.data_sources.csv_dataset_repository:save_dataset:79 - Dataset sauvegardé avec succès : /Users/surelmanda/Downloads/ml-projects/Clean-Architecture-MLops/health_lifestyle_diabetes/data/input/train.csv
2026-01-08 02:45:57.060 | INFO     | __main__:<module>:30 - Train dataset successfully saved at '/Users/surelmanda/Downloads/ml-projects/Clean-Architecture-MLops/health_lifestyle_diabetes/data/input/train.csv'
2026-01-08 02:45:57.061 | INFO     | __main__:<module>:22 - Saving test dataset | rows=14595 | columns=31
2026-01-08 02:45:57.061 | I

In [ ]:
from pandas import DataFrame


def summarize_dataset(df: DataFrame, max_examples: int = 5) -> DataFrame:
    """
    Fournit une synthèse détaillée du dataset :
    - Nom de colonne
    - Type de données
    - Nombre et pourcentage de valeurs manquantes
    - Cardinalité (nombre de valeurs uniques)
    - Exemples représentatifs

    Args:
        df (pd.DataFrame): Le DataFrame à analyser.
        max_examples (int): Nombre maximum d'exemples à afficher par colonne.

    Returns:
        pd.DataFrame: Tableau récapitulatif des colonnes et de leurs caractéristiques.
    """
    logger = LoguruLogger()

    if df is None or df.empty:
        logger.warning("Aucun dataset fourni ou dataset vide.")
        return DataFrame(columns=["Column", "Type", "Missing", "% Missing", "Cardinality", "Examples"])

    logger.info("Résumé détaillé du dataset en cours d'analyse...")
    total_rows = len(df)
    column_details = []

    for col in df.columns:
        col_type = df[col].dtype

        # Valeurs manquantes
        missing_count = df[col].isna().sum()
        missing_pct = round((missing_count / total_rows) * 100, 2)

        # Cardinalité
        cardinality = df[col].nunique(dropna=True)

        # Exemples représentatifs
        unique_values = df[col].dropna().unique()
        if df[col].dtype == "object" or df[col].dtype.name == "category":
            examples = unique_values[:max_examples]
        else:
            examples = sorted(unique_values[:max_examples])

        column_details.append([
            col,
            col_type,
            missing_count,
            missing_pct,
            cardinality,
            examples
        ])

    # Création du DataFrame récapitulatif
    summary_df = DataFrame(
        column_details,
        columns=["Column", "Type", "Missing", "% Missing", "Cardinality", "Examples"]
    ).sort_values(by="% Missing", ascending=False)

    # Logs synthétiques
    logger.info(f"Résumé terminé : {len(summary_df)} colonnes, {total_rows} lignes.")

    return summary_df




print("Affichage des 5 premières lignes du dataset d'entraînement enrichi :")
# display(train_df_enriched.head())
display(train_df.describe())

# En supposant que 'data' est votre DataFrame
display(train_df.dtypes)
summarize_data = summarize_dataset(train_df, 3)

display(summarize_data)

Affichage des 5 premières lignes du dataset d'entraînement enrichi :


,Age,alcohol_consumption_per_week,physical_activity_minutes_per_week,diet_score,sleep_hours_per_day,screen_time_hours_per_day,family_history_diabetes,hypertension_history,cardiovascular_history,bmi,...,cholesterol_total,hdl_cholesterol,ldl_cholesterol,triglycerides,glucose_fasting,glucose_postprandial,insulin_level,hba1c,diabetes_risk_score,diagnosed_diabetes
count,82702.000000,82702.000000,82702.000000,82702.000000,82702.000000,82702.000000,82702.000000,82702.000000,82702.000000,82702.000000,...,82702.000000,82702.000000,82702.000000,82702.000000,82702.000000,82702.000000,82702.000000,82702.000000,82702.000000,82702.000000
mean,50.163841,2.008718,118.877572,5.996520,6.996503,6.000937,0.220019,0.250840,0.079176,25.608891,...,185.930280,54.068414,102.939385,121.384501,111.140517,160.018440,9.071580,6.520899,30.215115,0.600034
std,15.488830,1.420623,84.286669,1.782218,1.094331,2.469720,0.414262,0.433499,0.270015,3.592292,...,32.002744,10.274665,33.369530,43.349357,13.588030,30.962219,4.971388,0.814468,9.067717,0.489894
min,19.000000,0.000000,0.000000,0.000000,3.000000,0.500000,0.000000,0.000000,0.000000,15.000000,...,100.000000,20.000000,50.000000,30.000000,60.000000,70.000000,2.000000,4.000000,2.700000,0.000000
25%,38.000000,1.000000,57.000000,4.800000,6.300000,4.300000,0.000000,0.000000,0.000000,23.200000,...,164.000000,47.000000,78.000000,91.000000,102.000000,139.000000,5.090000,5.970000,23.800000,0.000000
50%,51.000000,2.000000,100.000000,6.000000,7.000000,6.000000,0.000000,0.000000,0.000000,25.600000,...,186.000000,54.000000,102.000000,121.000000,111.000000,160.000000,8.800000,6.520000,28.900000,1.000000
75%,61.000000,3.000000,160.000000,7.200000,7.700000,7.700000,0.000000,1.000000,0.000000,28.000000,...,207.000000,61.000000,126.000000,151.000000,120.000000,181.000000,12.460000,7.070000,35.600000,1.000000
max,90.000000,10.000000,833.000000,10.000000,10.000000,16.800000,1.000000,1.000000,1.000000,39.200000,...,318.000000,96.000000,263.000000,344.000000,172.000000,287.000000,32.220000,9.800000,67.200000,1.000000


Age                                     int64
gender                                 object
ethnicity                              object
education_level                        object
income_level                           object
employment_status                      object
smoking_status                         object
alcohol_consumption_per_week            int64
physical_activity_minutes_per_week      int64
diet_score                            float64
sleep_hours_per_day                   float64
screen_time_hours_per_day             float64
family_history_diabetes                 int64
hypertension_history                    int64
cardiovascular_history                  int64
bmi                                   float64
waist_to_hip_ratio                    float64
systolic_bp                             int64
diastolic_bp                            int64
heart_rate                              int64
cholesterol_total                       int64
hdl_cholesterol                   

2026-01-08 02:46:23.303 | INFO     | __main__:summarize_dataset:26 - Résumé détaillé du dataset en cours d'analyse...
2026-01-08 02:46:23.391 | INFO     | __main__:summarize_dataset:63 - Résumé terminé : 31 colonnes, 82702 lignes.


,Column,Type,Missing,% Missing,Cardinality,Examples
0,Age,int64,0,0.0,70,"[46, 65, 67]"
16,waist_to_hip_ratio,float64,0,0.0,40,"[0.85, 0.86, 0.87]"
29,diabetes_stage,object,0,0.0,5,"[Pre-Diabetes, Type 2, No Diabetes]"
28,diabetes_risk_score,float64,0,0.0,562,"[23.3, 35.7, 43.3]"
27,hba1c,float64,0,0.0,545,"[6.01, 7.42, 7.81]"
26,insulin_level,float64,0,0.0,2329,"[5.51, 9.5, 10.56]"
25,glucose_postprandial,int64,0,0.0,210,"[146, 191, 205]"
24,glucose_fasting,int64,0,0.0,108,"[104, 124, 133]"
23,triglycerides,int64,0,0.0,261,"[81, 131, 197]"
22,ldl_cholesterol,int64,0,0.0,188,"[68, 78, 148]"
